In [64]:
# Script to Scrape Responses to Email and piece together for NO SQL DB.

%load_ext autoreload
%autoreload 2
# import importlib
# importlib.reload()

import logging
import pandas as pd
from config import imap_password_customplanet, db_username, db_password
from modules.scrapes import scrape
from modules.db_operations import DatabaseConnector

email_address = 'team@customplanet.com'
email_pass = imap_password_customplanet


logging.basicConfig(filename='Email_Scraper.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S',force=True)


# ------------------------------------packaging process into final function----------------------------------------

def process(subject_line, email_address, email_pass):

    #create the msgs via scrape, then iterate through list of messages, and lastly cleanse frame
    email_address = 'team@customplanet.com'
    email_pass = imap_password_customplanet
    subject_line = 'Local Fall Basketball Jerseys'

    msgs_inbox = scrape.scrape_msgs_outbox_or_inbox('inbox', subject_line, email_address, email_pass)
    inbox, inbox_soup = scrape.create_msg_frame(msgs_inbox)
    inbox = scrape.cleanse_frame(inbox, 'inbox')
    inbox.name = 'inbox'

    msgs_outbox = scrape.scrape_msgs_outbox_or_inbox('outbox', subject_line, email_address, email_pass)
    outbox, outbox_soup = scrape.create_msg_frame(msgs_outbox)
    outbox = scrape.cleanse_frame(outbox, 'outbox')
    outbox.name = 'outbox'

    thread = scrape.piece_together(outbox, inbox)
    
    # Specify your database connection details
    server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
    database = 'emailcampaign'
    db = 'emailcampaign'
    table_name = 'thread'

    # Instantiate the DatabaseConnector class
    db_connector = DatabaseConnector(server, database, db_username, db_password, db, table_name)

    #If there are any new records, append them onto thread table
    #Update the reply_thread everytime
    new_records = db_connector.append_new_records(thread)
    db_connector.update_reply_thread(thread)

    return(inbox, outbox, thread, new_records)

# The foreign constraint enforces that this must have been sent initially. 
inbox, outbox, thread, new_records = process('Local Fall Basketball Jerseys', email_address, email_pass)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
You have 1 email responses to the specified search criteria Local Fall Basketball Jerseys in the inbox
You have 2 email responses to the specified search criteria Local Fall Basketball Jerseys in the outbox


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


No new records to send


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


Reply_Thread updated
Reply_Thread updated


# To DO

In [ ]:
#Properly get first_message. 
# This should occur in the outbox, hypothesis....

#Create PNGs on hoodies images using Selenium. 


#Create a seperate module for parsing the body of text for NLP
#Scrape out the body at a later date for sentiment analysis
#IF inbox is empty default to just outbox

#Look into this at some point
#Why with the reply of 'Y' does it have an empty field on the reply_thread

#Thirdly, what if the email is not directly replied to. In that case. 
# The scenarion would be the following, email campaign sent out to sammytaylor2006@yahoo.com
#The customer indirectly reaches out to CP, and not through the thread. This calls for a seperate table.
# and a seperate scrape 

#Integrate SSIS at some point for views


#Integrating prior orders and linking them in a SQL view.

#Predictive analytics based on prior orders

#Figure out why the run time is so long